In [1]:
!pip install elasticsearch

     |████████████████████████████████| 378 kB 14.3 MB/s 


In [2]:
import os
import sys
import json

import elasticsearch
from elasticsearch import Elasticsearch
import pandas as pd

In [19]:
# 엘라스틱서치 오브젝트 만들기
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [20]:
# connection check
es.ping()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


True

In [22]:
# 인덱스 만들기
es.indices.create(index='my-foo', ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'my-foo'}

In [23]:
# 인덱스 보기
res = es.indices.get_alias("*")
for Name in res:
  print(Name)

.apm-custom-link
.apm-agent-configuration
.kibana_7.15.1_001
.tasks
.kibana_task_manager_7.15.1_001
my-foo
.kibana-event-log-7.15.1-000001


/tmp/ipykernel_10128/1551562474.py:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.indices.get_alias("*")
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: this request accesses system indices: [.apm-custom-link, .apm-agent-configuration, .kibana_7.15.1_001, .tasks, .kibana_task_manager_7.15.1_001], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)


In [25]:
# 인덱스 삭제
es.indices.delete(index='wiki_documents', ignore=[400, 404])

{'acknowledged': True}

# Prepare data for Elastic search

In [9]:
with open('../data/wikipedia_documents.json') as f:
  data = json.load(f)

In [10]:
count = 0
for k,v in data.items():
  print(k, v)
  count += 1
  if count == 1:
    break

0 {'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.', 'corpus_source': '위키피디아', 'url': 'TODO', 'domain': None, 'title': '나라 목록', 'author': None, 'html': None, 'document_id': 0}


In [11]:
len(data)

60613

In [26]:
df = pd.read_json('../data/wikipedia_documents.json').T

In [27]:
df.head()

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4


In [28]:
df.nunique()

text             56737
corpus_source        1
url                  1
domain               0
title            31755
author               0
html                 0
document_id      60613
dtype: int64

### 중복 text 레코드 제거

In [29]:
df = df.drop_duplicates(subset=['text'])
df.nunique()

text             56737
corpus_source        1
url                  1
domain               0
title            31726
author               0
html                 0
document_id      56737
dtype: int64

In [30]:
len(df)

56737

In [31]:
print(df.loc[df['document_id']==23174]['text'])

23174    순항 유도탄에 적용되는 추진기관은 일반적으로 생각하는 로켓이 사용되지 않는다. 이는...
Name: text, dtype: object


In [32]:
df2 = df.to_dict('records')

In [33]:
df2[0]

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '나라 목록',
 'author': None,
 'html': None,
 'document_id': 0}

In [34]:
def generator(df2):
  for c, line in enumerate(df2):
    yield {
      '_index': 'wiki_documents',
      '_type': '_doc',
      '_id': line.get('document_id', None),
      '_source': {
        'title': line.get('title', ''),
        'text': line.get('text', '')
      }
    }
  raise StopIteration

In [35]:
wiki_generator = generator(df2)

In [36]:
wiki_generator

<generator object generator at 0x7f1739b873c0>

# Settings and Mappings

In [37]:
# Settings = {
#   'settings':{
#     'number_of_shards':1,
#     'number_of_replicas':1,
#   },
#   'mappings':{
#     'properties':{
#       'title':{
#         'type':'text'
#       },
#       'text':{
#         'type':'text'
#       }
#     }
#   }
# }
CONFIG = {
  'settings':{
    'number_of_shards':1,
    'number_of_replicas':1,
    'analysis':{
      'analyzer': {
        'nori_analyzer':{
          'type': 'custom',
          'tokenizer': 'nori_tokenizer',
          'filter':[
            'nori_part_of_speech_filter'
          ]
        }
      },
      'filter':{
        'nori_part_of_speech_filter': {
          'type': 'nori_part_of_speech',
        }
      }
    }
  },
  'mappings':{
    'properties':{
      'title':{
        'type':'text'
      },
      'text':{
        'type':'text',
        "analyzer": "nori_analyzer"
      }
    }
  }
}


In [38]:
es.indices.create(index='wiki_documents', ignore=[400,404], body=CONFIG)

/tmp/ipykernel_10128/2280445778.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index='wiki_documents', ignore=[400,404], body=CONFIG)
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki_documents'}

# Upload the Data into Elastic Search

In [39]:
from elasticsearch import helpers

In [40]:
try:
  res = helpers.bulk(es, generator(df2))
  # print(res)
except Exception as e:
  print(e)
  print('Done')

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


generator raised StopIteration
Done


In [45]:
res

{'wiki_documents': {'aliases': {}},
 '.apm-custom-link': {'aliases': {}},
 '.kibana-event-log-7.15.1-000001': {'aliases': {'.kibana-event-log-7.15.1': {'is_write_index': True}}},
 '.kibana_7.15.1_001': {'aliases': {'.kibana': {}, '.kibana_7.15.1': {}}},
 '.apm-agent-configuration': {'aliases': {}},
 '.tasks': {'aliases': {}},
 '.kibana_task_manager_7.15.1_001': {'aliases': {'.kibana_task_manager': {},
   '.kibana_task_manager_7.15.1': {}}}}

# queries

In [46]:
es.count(index = 'wiki_documents')


{'count': 56500,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [47]:
es.get(index='wiki_documents', id=0)

{'_index': 'wiki_documents',
 '_type': '_doc',
 '_id': '0',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 'found': True,
 '_source': {'title': '나라 목록',
  'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'}}

In [46]:
es.search(index='wiki_documents', body={'size':1, 'query':{'match':{'text':'국제적인 승인을 널리'}}})

/tmp/ipykernel_6464/706394350.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index='wiki_documents', body={'size':1, 'query':{'match':{'text':'국제적인 승인을 널리'}}})
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'took': 35,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1775, 'relation': 'eq'},
  'max_score': 23.013771,
  'hits': [{'_index': 'wiki_documents',
    '_type': '_doc',
    '_id': '0',
    '_score': 23.013771,
    '_source': {'title': '나라 목록',
     'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'}}]}}

In [17]:
es.indices.get_mapping(index='wiki_documents')

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'wiki_documents': {'mappings': {'properties': {'text': {'type': 'text'},
    'title': {'type': 'text'}}}}}

In [20]:
settings = {
  'settings':{
    'number_of_shards':1,
    'number_of_replicas':1,
    'analysis':{
      'analyzer': {
        'nori_analyzer':{
          'type': 'custom',
          'tokenizer': 'nori'
        }
      }
    }
  }
}

In [ ]:
es.indices.put_settings(body=settings, index='wiki_documents')

In [18]:
mapping = {
'mappings':{
    'properties':{
      'title':{
        'type':'text'
      },
      'text':{
        'type':'text',
        "analyzer": "nori_analyzer"
      }
    }
  }
}

In [ ]:
es.indices.put_mapping(body=mapping, index='wiki_documents')

In [16]:
es.indices.analyze(
  body={
    'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.',
    'analyzer':'nori',
  }
)

{'tokens': [{'token': '문서',
   'start_offset': 2,
   'end_offset': 4,
   'type': 'word',
   'position': 1},
  {'token': '나라',
   'start_offset': 6,
   'end_offset': 8,
   'type': 'word',
   'position': 3},
  {'token': '목록',
   'start_offset': 9,
   'end_offset': 11,
   'type': 'word',
   'position': 4},
  {'token': '이',
   'start_offset': 11,
   'end_offset': 12,
   'type': 'word',
   'position': 5},
  {'token': '세계',
   'start_offset': 17,
   'end_offset': 19,
   'type': 'word',
   'position': 8},
  {'token': '206',
   'start_offset': 20,
   'end_offset': 23,
   'type': 'word',
   'position': 9},
  {'token': '개',
   'start_offset': 23,
   'end_offset': 24,
   'type': 'word',
   'position': 10},
  {'token': '나라',
   'start_offset': 25,
   'end_offset': 27,
   'type': 'word',
   'position': 11},
  {'token': '현황',
   'start_offset': 31,
   'end_offset': 33,
   'type': 'word',
   'position': 14},
  {'token': '주권',
   'start_offset': 35,
   'end_offset': 37,
   'type': 'word',
   'position

In [88]:
es.search(
  index='wiki_documents', 
  body={
    'query':{
      'match':{
        'text':'용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?',
        # 'tokenizer': 'nori'
      }
    }
  },
  size=3
)

/tmp/ipykernel_11168/1677013031.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(


{'took': 19,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 21.447016,
  'hits': [{'_index': 'wiki_documents',
    '_type': '_doc',
    '_id': '20081',
    '_score': 21.447016,
    '_source': {'title': '크리스 니코스키',
     'text': 'SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 좌완 선발 투수가 필요하여 니코스키에게 관심을 두고 있었던 두산 베어스가 SK 와이번스에 계약 양도를 신청하여 곧바로 맷 왓슨의 대체 용병으로 두산 베어스로 이적했고 두산 베어스의 선발진에 합류하여 비로소 첫 승을 따 냈으며  승리를 대부분 한화 이글스전에서 기록하여 한화 이글스에는 강력한 천적으로 자리매김했다.  그러나 예전부터 주로 원포인트 및 중간계투로 뛰다가 선발로 전환한 탓에 선발로는 아직 익숙하지 않아 3회까지 투구수가 70개 가까이 되는 경기가 대부분이라는 단점이 있었다.  2009년 9월 13일 KIA 타이거즈전에서 비로소 초반에 급격하게 늘어나는 투구수를 극복하여 승리를 따 내기도 했다.  두산 베어스 합류 후 12경기에 더 나서고 2009년 정규 시즌을 4승 8패, 평균 자책 3.78로 마무리했다. 완전한 선발 전업 후 후반기에 조금씩 나아지는 모습을 보였으나, 잠실야구장에서 롯데 자이언츠와 맞붙었던 2009년 9월 29일 준 플레이오프 1차전에서 무실점으로 3이닝을 호투하던 중 갑작스러운 어깨 통증으로 조기 강판당했고, 그 날 두산 베어스는 롯데 자이언츠에게 1차전을 내주었다. 진단 결과 극상근 손상 판정을 받아 포스트 시즌 전력에서 완전히 이탈하고 말았다. 

In [5]:
res = es.search(
  index='wiki_documents', 
  body={
    'query':{
      'match':{
        'text':'용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?',
        # 'tokenizer': 'nori'
      }
    }
  },
  size=3
)

/tmp/ipykernel_10128/1528268646.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(


In [13]:
res['hits']['hits'][0]['_source']['text']

'SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 좌완 선발 투수가 필요하여 니코스키에게 관심을 두고 있었던 두산 베어스가 SK 와이번스에 계약 양도를 신청하여 곧바로 맷 왓슨의 대체 용병으로 두산 베어스로 이적했고 두산 베어스의 선발진에 합류하여 비로소 첫 승을 따 냈으며  승리를 대부분 한화 이글스전에서 기록하여 한화 이글스에는 강력한 천적으로 자리매김했다.  그러나 예전부터 주로 원포인트 및 중간계투로 뛰다가 선발로 전환한 탓에 선발로는 아직 익숙하지 않아 3회까지 투구수가 70개 가까이 되는 경기가 대부분이라는 단점이 있었다.  2009년 9월 13일 KIA 타이거즈전에서 비로소 초반에 급격하게 늘어나는 투구수를 극복하여 승리를 따 내기도 했다.  두산 베어스 합류 후 12경기에 더 나서고 2009년 정규 시즌을 4승 8패, 평균 자책 3.78로 마무리했다. 완전한 선발 전업 후 후반기에 조금씩 나아지는 모습을 보였으나, 잠실야구장에서 롯데 자이언츠와 맞붙었던 2009년 9월 29일 준 플레이오프 1차전에서 무실점으로 3이닝을 호투하던 중 갑작스러운 어깨 통증으로 조기 강판당했고, 그 날 두산 베어스는 롯데 자이언츠에게 1차전을 내주었다. 진단 결과 극상근 손상 판정을 받아 포스트 시즌 전력에서 완전히 이탈하고 말았다. 플레이오프가 끝난 이후 이닝 소화 능력이 떨어진다는 이유로 두산 베어스가 재계약을 포기하면서 대한민국을 떠나게 되었다.  두산에서의 마지막이 좋지 않았지만, 그가 두산에 입단하여 2군에서 불펜 피칭을 했을 때 그를 지켜보았던 박종훈 당시 2군 감독은 니코스키를 두고 성격이 좋았다고 하였다.'

In [17]:
from datasets import Dataset, DatasetDict

df = pd.DataFrame({'질문':['질문1', '질문2', '질문3'], '답변':['답변1', '답변2', '답변3']})
DatasetDict({"validation": Dataset.from_pandas(df)})

DatasetDict({
    validation: Dataset({
        features: ['질문', '답변'],
        num_rows: 3
    })
})